# Add Embeddings to a Run

## ADD LINKS

## ADD DESCRIPTION

In [1]:
PROJECT_NAME = "Bounding Box Classifier"
TRANSIENT_DATA_PATH = "../../transient_data"
BATCH_SIZE = 32
DATASET_NAME = "Balloons"
RUN_NAME = "Embeddings collection"

## Imports

In [2]:
import tlc
import numpy as np

## Load Table and Embeddings

In [3]:
# Load the table created in the first notebook
original_table = tlc.Table.from_names(
    table_name="original", 
    dataset_name=DATASET_NAME, 
    project_name=PROJECT_NAME
)

# Load the embeddings created in the second notebook
embeddings = np.load(f"{TRANSIENT_DATA_PATH}/bb_classifier_embeddings.npy")

In [11]:
run = tlc.init(project_name=PROJECT_NAME, run_name=RUN_NAME)

embedding_list_schema = tlc.Schema(
    display_name="Embeddings",
    writable=False,
    value=tlc.Float32Value(number_role=tlc.NUMBER_ROLE_XYZ_COMPONENT),
    size0=tlc.DimensionNumericValue(value_min=3, value_max=3, number_role=tlc.NUMBER_ROLE_XYZ_COMPONENT, enforce_max=True),
    size1=tlc.DimensionNumericValue(enforce_min=False, value_max=1000),
)

metrics_writer = tlc.MetricsTableWriter(run_url=run.url, foreign_table_url=original_table.url, column_schemas={"embeddings": embedding_list_schema})

embedding_index = 0
for i, sample in enumerate(original_table):
    embeddings_list = []
    example_ids = []
    for bb in sample["bbs"]["bb_list"]:
        embeddings_list.append(embeddings[embedding_index].tolist())
        embedding_index += 1

    metrics_writer.add_row(
        {
            "embeddings": [[e[0] for e in embeddings_list], [e[1] for e in embeddings_list], [e[2] for e in embeddings_list]],
            tlc.EXAMPLE_ID: i
        }
    )

# Finalize writes the metrics table to disk
metrics_table = metrics_writer.finalize()

# Ensure the metrics table is associated with the run
run.add_metrics_table(metrics_table)

# Mark the run as completed
run.set_status_completed()

WARNING 3lc: Blacklisting Url: s3://3lc-projects due to EndpointConnectionError, Could not connect to the endpoint URL: "https://3lc-projects.s3.eu-north-1.amazonaws.com/?list-type=2&prefix=cars%2Fruns%2F&delimiter=%2F&encoding-type=url". Will retry in 0:00:20.
WARNING 3lc: Blacklisting Url: s3://zephyr-experiments/projects due to EndpointConnectionError, Could not connect to the endpoint URL: "https://zephyr-experiments.s3.eu-north-1.amazonaws.com/?list-type=2&prefix=projects%2FZephyr-7B-beta_subset%2Fdatasets%2Fultrachat_200k-train%2Ftables%2Foriginal%2F&delimiter=%2F&encoding-type=url". Will retry in 0:00:20.
WARNING 3lc: Blacklisting Url: s3://zephyr-experiments/projects due to EndpointConnectionError, Could not connect to the endpoint URL: "https://zephyr-experiments.s3.eu-north-1.amazonaws.com/?list-type=2&prefix=projects%2F&delimiter=%2F&encoding-type=url". Will retry in 0:00:20.
WARNING 3lc: Blacklisting Url: s3://3lc-esmart-systems-data/tables due to EndpointConnectionError, C